In [ ]:
# 보팅 베깅 부스팅 



앙상블 


보팅, 배깅, 부스팅

하드보팅은 다수의 클래시파이의 결과 값

소프트 보팅은 클래시 파이들의 class  확률을 평균하여 결정 

# 앙상블(Ensemble) 학습

In [1]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

In [3]:
# 스케일링 필요
from sklearn.preprocessing import MinMaxScaler
cancer_scaled = MinMaxScaler().fit_transform(cancer.data)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    cancer_scaled, cancer.target, stratify=cancer.target, test_size=0.2, random_state=2021
)

### 1. 앙상블 학습 - Hard Vorting
 - 로지스틱 회귀
 - 서포트 벡터 머신
 - K최근접 이웃 (KNN , K Nearest Neighbor)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [12]:
lrc = LogisticRegression()
svc = SVC()
knn = KNeighborsClassifier()

In [13]:
from sklearn.ensemble import VotingClassifier

voc = VotingClassifier(
    estimators=[('LRC',lrc), ('SVC', svc), ('KNN', knn)], voting='hard'   # 값을 튜플로 준다.
)

In [14]:
voc.fit(X_train, y_train)
voc.score(X_test, y_test)

0.9824561403508771

* 개별 분류기의 성능

In [15]:
lrc.fit(X_train, y_train)
svc.fit(X_train, y_train)
knn.fit(X_train, y_train)
lrc.score(X_test, y_test), svc.score(X_test, y_test), knn.score(X_test, y_test)

# 다 똑같이 나와 재미없어

(0.9824561403508771, 0.9824561403508771, 0.9824561403508771)

### 2. 앙상블 학습 - 소프트 보팅

In [16]:
# 메소드와 속성 
print(dir(lrc))

# C 하이퍼 파라메터 , __   __system관련, __gt__ , __lt__ 비교 애들,
# 인잇. 겟파람. 절편 intercept, predict_proba
# 프레딕스 프로바 ( 확률)

['C', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_n_features', '_estimator_type', '_get_param_names', '_get_tags', '_more_tags', '_predict_proba_lr', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_validate_data', 'class_weight', 'classes_', 'coef_', 'decision_function', 'densify', 'dual', 'fit', 'fit_intercept', 'get_params', 'intercept_', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_features_in_', 'n_iter_', 'n_jobs', 'penalty', 'predict', 'predict_log_proba', 'predict_proba', 'random_state', 'score', 'set_params', 'solver', 'sparsify', 'tol', 'verbose', 'warm_start']


In [17]:
lrc.predict_proba(X_test[:5])   # 하드보팅을 분류, 0,1 # 소프트 보팅은 확률 이용 프레딕트 프로바 중요

array([[0.36348222, 0.63651778],
       [0.97162943, 0.02837057],
       [0.1869565 , 0.8130435 ],
       [0.02133462, 0.97866538],
       [0.0548391 , 0.9451609 ]])

In [18]:
lrc.predict(X_test[:5])   # 위에서 0일 확률 1일 확률. 확률 큰 쪽으로 잘 예측. 

array([1, 0, 1, 1, 1])

In [ ]:
# svc.predict_proba(X_test[:5])  # svc도 프레딕트 프로바 - 소프트 보팅 가능하나. 어트리뷰트 에러

# predict_proba is not available when  probability=False 에러
# 왓더??

내가 학습을 시킬때 프로바빌리티를 True 로 놔주면 할 수 있어 .

In [21]:
svc.get_params()
# 'probability': False,

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [22]:
svc2 = SVC(probability=True)
svc2.fit(X_train, y_train)
svc2.predict_proba(X_test[:5])

array([[4.60223539e-01, 5.39776461e-01],
       [9.99833684e-01, 1.66316421e-04],
       [1.49527856e-02, 9.85047214e-01],
       [3.15409458e-03, 9.96845905e-01],
       [4.55061793e-03, 9.95449382e-01]])

In [23]:
knn.predict_proba(X_test[:5])

array([[0.4, 0.6],
       [1. , 0. ],
       [0.2, 0.8],
       [0. , 1. ],
       [0. , 1. ]])

In [24]:
voc2 = VotingClassifier(
    estimators=[('LRC',lrc), ('SVC', svc2), ('KNN', knn)], voting='soft'   # 값을 튜플로 준다. 
)

In [25]:
voc2.fit(X_train, y_train)
voc2.score(X_test, y_test) # 뭔가 상승 근데 이 값을 뭐라고 불러 분류성능? 1쪽에 가까운겨? 
# 응망진망마진창.

0.9912280701754386

In [26]:
voc2.predict_proba(X_test[:5])
# voc2는 앙상블, 아까 3개의 값을 평균낸 확률이 될것. 

array([[0.41337353, 0.58662647],
       [0.99050503, 0.00949497],
       [0.13361674, 0.86638326],
       [0.00803581, 0.99196419],
       [0.01963058, 0.98036942]])

### * GridSearchCV 적용 . 파라메터 튜닝어 어디에 전달. ? 에스티메티 3개나 되는데

In [27]:
params = {
    'LRC__C' : [0.1, 1, 10],
    'SVC__C' : [0.1, 1, 10]
}

In [28]:
from sklearn.model_selection import GridSearchCV
grid_voc2 = GridSearchCV(voc2, param_grid=params, scoring='accuracy', cv=3)
grid_voc2.fit(X_train, y_train)
grid_voc2.best_params_

{'LRC__C': 0.1, 'SVC__C': 10}

In [29]:
# 베스트 파라메터 이용
grid_voc2.best_estimator_.score(X_test, y_test)

0.9912280701754386

In [30]:
# 튜닝욕심

In [31]:
params = {
    'LRC__C' : [0.01, 0.05, 0.1, 0.2, 0.5],
    'SVC__C' : [5,10,20,50]
}

grid_voc2 = GridSearchCV(voc2, param_grid=params, scoring='accuracy', cv=3)
grid_voc2.fit(X_train, y_train)
grid_voc2.best_params_

{'LRC__C': 0.05, 'SVC__C': 50}